In [1]:
!pip install biopython

In [3]:
import scanpy as sc 

In [4]:
import os

In [5]:
import numpy as np
import pandas as pd

In [4]:
from Bio import Entrez
from Bio import SeqIO

In [17]:
import subprocess

In [51]:
import re
from itertools import count

In [8]:
os.chdir("/scratch/users/sschulz/neuron_data")

In [9]:
os.getcwd()

'/scratch/users/sschulz/neuron_data'

In [10]:
adata = sc.read_h5ad("/scratch/users/sschulz/neuron_data/deep_layer_non_it_excitatory_neurons.h5ad")

In [11]:
# to test if it works just do it on 10 cells
adata = adata[adata.obs_names[0:10]]

In [31]:
fasta_path = "/scratch/users/sschulz/neuron_data/Homo_sapiens.GRCh38.cdna.all.fa"

In [18]:
super_cluster_term = "deep_layer_non_it_excitatory_neurons"
dir_name = super_cluster_term + "_fastas"
subprocess.call(["mkdir", "-p", dir_name])

0

In [19]:
fasta_fn =  super_cluster_term + '.fa' 

In [ ]:
string_to_write = "" #this string is gonna be REALLY long...

for cell_index in range(0, len(adata.obs_names)):
    print("The current cell index is {}".format(cell_index))
    cell_name = adata.obs_names[cell_index]

    string_to_write += (super_cluster_term +  "_" + cell_name + "_start")
    string_to_write += "\n"
    
    for gene_index in range(0, len(adata.var_names)):
        print("The current gene_index is {}".format(gene_index))
        read_count = int(adata.X[cell_index, gene_index])
        ensembl_id = "gene:" + adata.var_names[gene_index]
        if read_count > 0:
            grepped_transcript_string = subprocess.run(["grep", "-A1", ensembl_id, fasta_path], capture_output=True)
            grepped_transcript_string = grepped_transcript_string.stdout.decode()
            if grepped_transcript_string == '':
                print("ensembl id not found in supplied fasta file, skipping")
                continue
            copied_transcript_strings = "\n".join([grepped_transcript_string]*read_count)
           
            #want to use regex to make every ensembl_id header have a unique id
            start_index = 0
            counter = count(start_index)
            return_string = re.sub(r"{}".format(ensembl_id), lambda x: x.group(0) + "_read_" + str(next(counter)), copied_transcript_strings) 

            string_to_write += return_string
            string_to_write += "\n"
        else:
            print("no reads associated with this transcript")
            
    string_to_write +=  (super_cluster_term +  "_" + cell_name + "_end")
    
with open(fasta_fn, 'w') as f:
    f.write(string_to_write)
subprocess.call(["mv", fasta_fn, dir_name])

The current cell index is 0
The current gene_index is 0
no reads associated with this transcript
The current gene_index is 1
no reads associated with this transcript
The current gene_index is 2
ensembl id not found in supplied fasta file, skipping
The current gene_index is 3
no reads associated with this transcript
The current gene_index is 4
no reads associated with this transcript
The current gene_index is 5
ensembl id not found in supplied fasta file, skipping
The current gene_index is 6
no reads associated with this transcript
The current gene_index is 7
no reads associated with this transcript
The current gene_index is 8
no reads associated with this transcript
The current gene_index is 9
no reads associated with this transcript
The current gene_index is 10
no reads associated with this transcript
The current gene_index is 11
no reads associated with this transcript
The current gene_index is 12
no reads associated with this transcript
The current gene_index is 13
The current gene_

In [46]:
grepped_transcript_string.stdout.decode()

''

In [60]:
adata.var['feature_biotype']

gene
ENSG00000233576    gene
ENSG00000121410    gene
ENSG00000268895    gene
ENSG00000148584    gene
ENSG00000175899    gene
                   ... 
ENSG00000203995    gene
ENSG00000162378    gene
ENSG00000159840    gene
ENSG00000074755    gene
ENSG00000036549    gene
Name: feature_biotype, Length: 29348, dtype: category
Categories (1, object): ['gene']

In [58]:
grepped_transcript_string

'>ENST00000418937.6 cdna chromosome:GRCh38:12:125065434:125107162:1 gene:ENSG00000081760.17 gene_biotype:protein_coding transcript_biotype:nonsense_mediated_decay gene_symbol:AACS description:acetoacetyl-CoA synthetase [Source:HGNC Symbol;Acc:HGNC:21298]\nCAGTCCCTTGTTCCCCGCCGCCGCCGTCGCTGACCCAGCCCGCCAGGCGCTCCTGACCGT\n--\n>ENST00000316519.11 cdna chromosome:GRCh38:12:125065435:125143316:1 gene:ENSG00000081760.17 gene_biotype:protein_coding transcript_biotype:protein_coding gene_symbol:AACS description:acetoacetyl-CoA synthetase [Source:HGNC Symbol;Acc:HGNC:21298]\nAGTCCCTTGTTCCCCGCCGCCGCCGTCGCTGACCCAGCCCGCCAGGCGCTCCTGACCGTC\n--\n>ENST00000537477.5 cdna chromosome:GRCh38:12:125091013:125107154:1 gene:ENSG00000081760.17 gene_biotype:protein_coding transcript_biotype:protein_coding_CDS_not_defined gene_symbol:AACS description:acetoacetyl-CoA synthetase [Source:HGNC Symbol;Acc:HGNC:21298]\nATTTGAGTGAAGAGCTGCAGGATGAGCGCACACTGAGTGGAGACTGCTGGTGTGCGTGGA\n--\n>ENST00000537564.5 cdna chromosome:GR

Old way of doing it

In [11]:
#to test, just do 10 cells for now
adata = adata[adata.obs_names[0:10]]

In [16]:
adata

View of AnnData object with n_obs × n_vars = 10 × 59357
    obs: 'roi', 'organism_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'sex_ontology_term_id', 'development_stage_ontology_term_id', 'donor_id', 'suspension_type', 'dissection', 'fraction_mitochondrial', 'fraction_unspliced', 'cell_cycle_score', 'total_genes', 'total_UMIs', 'sample_id', 'supercluster_term', 'cluster_id', 'subcluster_id', 'cell_type_ontology_term_id', 'tissue_ontology_term_id', 'is_primary_data', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'Biotype', 'Chromosome', 'End', 'Gene', 'Start', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'batch_condition', 'schema_version', 'title'
    obsm: 'X_UMAP', 'X_tSNE'

In [15]:
for super_cluster_term in np.unique(adata.obs["supercluster_term"]):
    dir_name = super_cluster_term + "_fastas"
    subprocess.call(["mkdir", "-p", dir_name])

In [ ]:
for cell_index in range(0, len(adata.obs_names)):
    print("The current cell index is {}".format(cell_index))
    cell_name = adata.obs_names[cell_index]
    super_cluster_term = adata.obs['supercluster_term'][cell_name]
    fasta_fn =  super_cluster_term + '_' + cell_name + '_cell_fasta.fa'
    with open(fasta_fn, 'w') as f:
        f.write("")
        for gene_index in range(0, len(adata.var_names)):
            print("The current gene_index is {}".format(gene_index))
            ensembl_id = adata.var_names[gene_index]
            f.write(ensembl_id)
            #os.system("grep -A1 {gene_index} ../data/ensembl_annotated_transcriptome.fa | grep -v {gene_index} >> {fasta_fn}")
            read_count = adata.X[cell_index, gene_index]
            print(gene_index)
            if read_count != 0:
                #soooooooooooooooooooooooooooooooooo slooooooooooooooooooooooooooooooooooooooooooooooooooooooooow
                #this grepping per read count shit can't be the right track, we should construct one big string by using
                #a search function from python or read lines or somethin, then append taht
                #string to a file when its all done. This can't be faster than that
                subprocess.call(["sh", "../src/create_cell_fasta.sh", ensembl_id, fasta_fn, str(int(read_count))])
                #these calls generate ALOT of files, should move all the files for a supercluster to their own folder
                #i was on the fence about keeping the files separate but i think generating a csv with normalized compression
                #distance values for all neurons is more interesting than making a weird text file with all of them and doing a sort of average
                #compression distance with that one, i think getting all of them is more useful

The current cell index is 0
The current gene_index is 0
0
The current gene_index is 1
1
The current gene_index is 2
2
The current gene_index is 3
3
The current gene_index is 4
4
The current gene_index is 5
5
The current gene_index is 6
6
The current gene_index is 7
7
The current gene_index is 8
8
New task added (id 9860).
New task added (id 9861).
The current gene_index is 9
9
The current gene_index is 10
10
The current gene_index is 11
11
The current gene_index is 12
12
New task added (id 9862).
New task added (id 9863).
The current gene_index is 13
13
The current gene_index is 14
14
The current gene_index is 15
15
New task added (id 9864).
New task added (id 9865).
The current gene_index is 16
16
The current gene_index is 17
17
The current gene_index is 18
18
New task added (id 9866).
New task added (id 9867).
The current gene_index is 19
19
The current gene_index is 20
20
New task added (id 9868).
New task added (id 9869).
The current gene_index is 21
21
New task added (id 9870).
Ne

In [ ]:
for super_cluster_term in np.unique(adata.obs["supercluster_term"]):
    subprocess.call(["sh", "../src/move_cell_fastas.sh", super_cluster_term])

In [20]:
np.array(adata.X[1:1].todense()).shape

(1, 59357)

In [52]:
test = os.system("grep -A1 ENST00000666593.2 ../data/ensembl_annotated_transcriptome.fa | grep -v ENST00000666593.2")

AGAGGCGGAGCCGCTGTGGCACTGCTGCGCCTCTGCTGCGCCTCGGGTGTCTTTTGCGGC


In [26]:
adata.X[50,1]

0.0